# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [244]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_df = (pd.read_csv(r'C:\Users\zarri\scaling_to_production\05_src\data\fires\forestfires.csv', header = 0, names = columns))

In [245]:
# Display the first few rows and check for any missing values
print(fires_df.head())
print(fires_df.info())  # Check data types and missing values

   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain  \
0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   
1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   
2        7        4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   
3        8        6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   
4        8        6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   

   area  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi    

In [246]:
print(fires_df.columns)

Index(['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi',
       'temp', 'rh', 'wind', 'rain', 'area'],
      dtype='object')


In [247]:
numeric_columns = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area']
fires_df[numeric_columns] = fires_df[numeric_columns].apply(pd.to_numeric, errors='coerce')


In [248]:

print(fires_df.dtypes)


coord_x      int64
coord_y      int64
month       object
day         object
ffmc       float64
dmc        float64
dc         float64
isi        float64
temp       float64
rh           int64
wind       float64
rain       float64
area       float64
dtype: object


# Get X and Y

Create the features data frame and target data.

In [249]:
# Separate features (X) and target (y)
X = fires_df.drop(columns=['area'])  # X contains all columns except 'area'
y = fires_df['area']  # y is the 'area' column (target variable)

# Check the shapes to ensure correct separation
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Shape of X: (517, 12)
Shape of y: (517,)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [250]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Define numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps into a single ColumnTransformer
preprocessor1 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [251]:
preprocessor1

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['month', 'day'])])

In [252]:

# Apply preprocessor1 to X
X_preprocessed = preprocessor1.fit_transform(X)

# Check the shape of the transformed data
print("Shape of X_preprocessed:", X_preprocessed.shape)


import pandas as pd
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=numeric_features + preprocessor1.transformers_[1][1]['onehot'].get_feature_names_out(categorical_features).tolist())

# Display the first few rows of the transformed DataFrame
print(X_preprocessed_df.head())


Shape of X_preprocessed: (517, 29)
    coord_x   coord_y      ffmc       dmc        dc       isi      temp  \
0  1.008313  0.569860 -0.805959 -1.323326 -1.830477 -0.860946 -1.842640   
1  1.008313 -0.244001 -0.008102 -1.179541  0.488891 -0.509688 -0.153278   
2  1.008313 -0.244001 -0.008102 -1.049822  0.560715 -0.509688 -0.739383   
3  1.440925  1.383722  0.191362 -1.212361 -1.898266 -0.004756 -1.825402   
4  1.440925  1.383722 -0.243833 -0.931043 -1.798600  0.126966 -1.291012   

         rh      wind      rain  ...  month_nov  month_oct  month_sep  \
0  0.411724  1.498614 -0.073268  ...        0.0        0.0        0.0   
1 -0.692456 -1.741756 -0.073268  ...        0.0        1.0        0.0   
2 -0.692456 -1.518282 -0.073268  ...        0.0        1.0        0.0   
3  3.233519 -0.009834  0.603155  ...        0.0        0.0        0.0   
4  3.356206 -1.238940 -0.073268  ...        0.0        0.0        0.0   

   day_fri  day_mon  day_sat  day_sun  day_thu  day_tue  day_wed  
0      1

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [253]:
from sklearn.preprocessing import PowerTransformer

numeric_transformer2 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('power_transform', PowerTransformer(method='yeo-johnson')) 
])

preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer2, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [254]:
preprocessor2

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler()),
                                                 ('power_transform',
                                                  PowerTransformer())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['month', 'day'])])

In [255]:
# Apply preprocessor2 to X
X_preprocessed2 = preprocessor2.fit_transform(X)

# Check the shape of X_preprocessed2
print(f"Shape of X_preprocessed2: {X_preprocessed2.shape}")

# Retrieve categorical feature names after one-hot encoding
onehot_columns = preprocessor2.named_transformers_['cat'].named_steps['onehot'].categories_
onehot_feature_names = []
for feature, categories in zip(categorical_features, onehot_columns):
    onehot_feature_names.extend([f"{feature}_{cat}" for cat in categories])

# Combine numeric feature names and one-hot encoded feature names
all_feature_names = numeric_features + onehot_feature_names

# Convert X_preprocessed2 to DataFrame for better inspection
X_preprocessed2_df = pd.DataFrame(X_preprocessed2, columns=all_feature_names)

# Print the first few rows and inspect the transformed data
print(X_preprocessed2_df.head())

# Check the data types to ensure everything is as expected
print(X_preprocessed2_df.dtypes)


Shape of X_preprocessed2: (517, 29)
    coord_x   coord_y      ffmc       dmc        dc       isi      temp  \
0  1.007649  0.604351 -1.325441 -1.454922 -1.543155 -0.969040 -1.725188   
1  1.007649 -0.198503 -0.416248 -1.268789  0.305688 -0.460641 -0.216826   
2  1.007649 -0.198503 -0.416248 -1.103929  0.450008 -0.460641 -0.769296   
3  1.427726  1.345115 -0.024215 -1.310966 -1.564699  0.166167 -1.710858   
4  1.427726  1.345115 -0.767015 -0.955637 -1.532800  0.307247 -1.257978   

         rh      wind      rain  ...  month_nov  month_oct  month_sep  \
0  0.596064  1.406894 -0.125368  ...        0.0        0.0        0.0   
1 -0.665368 -2.075371 -0.125368  ...        0.0        1.0        0.0   
2 -0.665368 -1.755114 -0.125368  ...        0.0        1.0        0.0   
3  2.321751  0.113191  7.976528  ...        0.0        0.0        0.0   
4  2.378413 -1.368546 -0.125368  ...        0.0        0.0        0.0   

   day_fri  day_mon  day_sat  day_sun  day_thu  day_tue  day_wed  
0      

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [256]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Define the regressors
baseline_regressor = LinearRegression()
advanced_regressor = RandomForestRegressor(random_state=42)


In [257]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline(steps=[
    ('preprocessor', preprocessor1),
    ('regressor', baseline_regressor)
])


In [258]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('regressor', baseline_regressor)
])


In [259]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline(steps=[
    ('preprocessor', preprocessor1),
    ('regressor', advanced_regressor)
])


In [260]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('regressor', advanced_regressor)
])


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [261]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Pipeline A (LinearRegression)
param_grid_A = {
    'regressor__fit_intercept': [True, False]
}

# Perform grid search with cross-validation
grid_search_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error')
grid_search_A.fit(X_train, y_train)

# Print best parameters and best score
print("Best parameters found for Pipeline A:")
print(grid_search_A.best_params_)
print("Best RMSE score found for Pipeline A:", np.sqrt(-grid_search_A.best_score_))


Best parameters found for Pipeline A:
{'regressor__fit_intercept': False}
Best RMSE score found for Pipeline A: 47.22683800403542


In [262]:
pipeline_B = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('regressor', baseline_regressor)
])

# Define parameter grid for Pipeline B
param_grid_B = {
    'regressor__fit_intercept': [True, False]
}

# Perform grid search with cross-validation for Pipeline B
grid_search_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error')
grid_search_B.fit(X_train, y_train)

# Print best parameters and best score for Pipeline B
print("Best parameters found for Pipeline B:")
print(grid_search_B.best_params_)
print("Best RMSE score found for Pipeline B:", np.sqrt(-grid_search_B.best_score_))


Best parameters found for Pipeline B:
{'regressor__fit_intercept': True}
Best RMSE score found for Pipeline B: 46.49781277004672


In [263]:
pipeline_C = Pipeline(steps=[
    ('preprocessor', preprocessor1),
    ('regressor', advanced_regressor)
])

# Define parameter grid for Pipeline C
param_grid_C = {
    'regressor__n_estimators': [50, 100, 200],  # Example hyperparameter to tune for RandomForestRegressor
    'regressor__max_depth': [None, 10, 20, 30]  # Example hyperparameter to tune for RandomForestRegressor
}

# Perform grid search with cross-validation for Pipeline C
grid_search_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error')
grid_search_C.fit(X_train, y_train)

# Print best parameters and best score for Pipeline C
print("Best parameters found for Pipeline C:")
print(grid_search_C.best_params_)
print("Best RMSE score found for Pipeline C:", np.sqrt(-grid_search_C.best_score_))


Best parameters found for Pipeline C:
{'regressor__max_depth': None, 'regressor__n_estimators': 100}
Best RMSE score found for Pipeline C: 52.76335102837149


In [264]:
pipeline_D = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('regressor', advanced_regressor)
])

# Define parameter grid for Pipeline D
param_grid_D = {
    'regressor__n_estimators': [50, 100, 200],  # Example hyperparameter to tune for RandomForestRegressor
    'regressor__max_depth': [None, 10, 20, 30]  # Example hyperparameter to tune for RandomForestRegressor
}

# Perform grid search with cross-validation for Pipeline D
grid_search_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error')
grid_search_D.fit(X_train, y_train)

# Print best parameters and best score for Pipeline D
print("Best parameters found for Pipeline D:")
print(grid_search_D.best_params_)
print("Best RMSE score found for Pipeline D:", np.sqrt(-grid_search_D.best_score_))


Best parameters found for Pipeline D:
{'regressor__max_depth': None, 'regressor__n_estimators': 100}
Best RMSE score found for Pipeline D: 52.14506462161665


# Evaluate

+ Which model has the best performance?

Pipeline B

# Export

+ Save the best performing model to a pickle file.

In [265]:
import pickle

In [266]:
with open('best_model_pipeline_B.pkl', 'wb') as file:
    pickle.dump(grid_search_B.best_estimator_, file)

In [267]:
with open('best_model_pipeline_B.pkl', 'rb') as file:
    loaded_pipeline = pickle.load(file)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [224]:
# Select an observation from the test set
X_test_selected = X_test.iloc[0]  


In [268]:
# Check data types in X_train
print(X_train.dtypes)


coord_x      int64
coord_y      int64
month       object
day         object
ffmc       float64
dmc        float64
dc         float64
isi        float64
temp       float64
rh           int64
wind       float64
rain       float64
dtype: object


In [269]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps into a single ColumnTransformer
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Assuming X_train and X_test are defined and include all features (numeric + categorical)
X_train_preprocessed = preprocessor2.fit_transform(X_train)
X_test_preprocessed = preprocessor2.transform(X_test)


In [270]:
print(X_train_preprocessed.dtype)

float64


In [271]:

categorical_feature_indices = preprocessor2.transformers_[1][1]['onehot'].get_feature_names_out()

# Create a list of all feature names including numeric and encoded categorical
feature_names = numeric_features + list(categorical_feature_indices)

# Convert NumPy array to DataFrame for easier inspection
import pandas as pd
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=feature_names)

# Check data types
print(X_train_preprocessed_df.dtypes)


coord_x      float64
coord_y      float64
ffmc         float64
dmc          float64
dc           float64
isi          float64
temp         float64
rh           float64
wind         float64
rain         float64
month_apr    float64
month_aug    float64
month_dec    float64
month_feb    float64
month_jan    float64
month_jul    float64
month_jun    float64
month_mar    float64
month_may    float64
month_nov    float64
month_oct    float64
month_sep    float64
day_fri      float64
day_mon      float64
day_sat      float64
day_sun      float64
day_thu      float64
day_tue      float64
day_wed      float64
dtype: object


In [ ]:
import shap

# Wrap the model with the SHAP explainer (assuming pipeline_B is your best-performing model)
explainer = shap.Explainer(pipeline_B.named_steps['regressor'], X_train_preprocessed)

# Select an observation from X_test
observation_index = 0  # Replace with the index of the observation you want to explain
X_observation = X_test_preprocessed[[observation_index]]  # Corrected indexing for NumPy array

# Calculate SHAP values for the selected observation
shap_values = explainer(X_observation)

# Plot SHAP values (example)
shap.plots.waterfall(shap_values[0])

Based on SHAP values, Feature 18 and 4 with the highest positive SHAP values of (+27.37 and +27.21 respectively) are likely the most important positive contributers to the prediction for the observation.

Feature 3 with large negative SHAP value of -19.75 is important negative contributor which means that its value reduces the prediction for the observation. Feature 21 with the negative SHAP value of -3.89 has the second negative contributor. 

To identify the features to remove we can look at features with consistently low SHAP values across many observations, as these features are likely contributing less to the model's predictions. Removing these features can simplify the model, and reduce overfitting and improve performance.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.